In [2]:
import json
import glob
from keras.models import model_from_json
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Convolution2D, MaxPooling2D, Dropout
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adamax
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization

np.random.seed(1)
tf.set_random_seed(1)
bands = 75
frames = 75
num_channels = 3
classes = 8

Using TensorFlow backend.


Label decoder without sklearn

In [3]:
def labels_decoder(labels):
    decoder = [
        'background',
        'bags',
        'door',
        'keyboard',
        'knocking_door',
        'ring',
        'speech',
        'tool'
    ]
    new_lb = []
    for lb in labels:
        new_lb.append(decoder[lb])
    return new_lb

In [4]:
def build_model():
    model = Sequential()

    # section 1

    model.add(Convolution2D(filters=32, kernel_size=5,
                            strides=2,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal",
                            input_shape=(frames, bands, num_channels)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Convolution2D(filters=32, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.3))

    # section 2
    model.add(Convolution2D(filters=64, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Convolution2D(filters=64, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # section 3
    model.add(Convolution2D(filters=128, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Convolution2D(filters=128, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Convolution2D(filters=128, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Convolution2D(filters=128, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # section 4
    model.add(Convolution2D(filters=512, kernel_size=3,
                            strides=1,
                            padding="valid",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Convolution2D(filters=512, kernel_size=1,
                            strides=1,
                            padding="valid",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # section 5
    model.add(Convolution2D(filters=8, kernel_size=1,
                            strides=1,
                            padding="valid",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(GlobalAveragePooling2D())

    model.add(Activation('softmax'))

    return model

In [5]:
def get_name(path):
    return path.split('/audio/')[1]

In [8]:
features = np.load('test_scaled.npy')
test_paths = np.sort(np.array(glob.glob('./data/test/audio/*.wav')))
y_paths = np.vectorize(get_name)(test_paths)
model = build_model()
weights = [
    ['weights/BC_d/BC_epoch_002_val_loss_0.6032.hdf5','BC_d.txt'],
    ['weights/BC_nd/model_weights30.hdf5','BC_nd.txt'],
    ['weights/regular/model_weights40.hdf5','regular.txt']
]
for w, path in weights:
    model.load_weights(w)
    y_test = model.predict(features)
    y = pd.concat([
        pd.DataFrame(y_paths),
        pd.DataFrame(np.amax(y_test, axis = 1)),
        pd.DataFrame(labels_decoder(np.argmax(y_test, axis = 1)))
    ],axis = 1)
    y.to_csv(path, header=None, index=None, sep='\t', mode='a')